<a href="https://colab.research.google.com/github/surodoot/Python/blob/main/%EB%8B%A8%EC%9D%BC%EC%B9%98%ED%99%98_%EC%95%94%ED%98%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import itertools as it
import pandas as pd
import re

In [ ]:
file = open('/content/drive/MyDrive/wikisent2.txt','r')
sentences = file.readlines()

In [ ]:
len(sentences)

In [ ]:
words = pd.read_csv('/content/drive/MyDrive/Words.csv')
words

In [ ]:
word_top = set(words['word'][:10000])

In [ ]:
random.shuffle(sentences)
sample_size = 1000
sample = sentences[0:sample_size]

In [ ]:
sample

In [ ]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'
alpha_freq = {}

for i in alphabet:
    alpha_freq[i] = 0

for sentence in sample:
    for i in sentence:
        i = i.lower()
        if alphabet.find(i) != -1:
            alpha_freq[i] += 1

plt.bar(range(26), alpha_freq.values())
plt.xticks(range(26), alphabet)
plt.show()

In [ ]:
alpha_rank = sorted(alpha_freq.keys(), key = lambda x : alpha_freq[x], reverse=True)

In [ ]:
print(alpha_rank)

In [ ]:
top_num = 100

alphabet3 = list(it.product(alphabet, repeat=3))
alphabet3 = [i[0]+i[1]+i[2] for i in alphabet3]

alpha3_freq = {}

for i in alphabet3:
    alpha3_freq[i] = 0

for sentence in sample:
    for i in range(len(sentence)-2):
        c = sentence[i] + sentence[i+1] + sentence[i+2]
        c = c.lower()
        if c[0] in alphabet and c[1] in alphabet and c[2] in alphabet:
            alpha3_freq[c] += 1

alpha3_top = set(sorted(alpha3_freq.keys(), key = lambda x : alpha3_freq[x], reverse=True)[:top_num])

In [ ]:
sorted(alpha3_freq.keys(), key = lambda x : alpha3_freq[x], reverse=True)[:top_num]

In [ ]:
alphabet4 = list(it.product(alphabet, repeat=4))
alphabet4 = [i[0]+i[1]+i[2]+i[3] for i in alphabet4]

alpha4_freq = {}

for i in alphabet4:
    alpha4_freq[i] = 0

for sentence in sample:
    for i in range(len(sentence)-3):
        c = sentence[i] + sentence[i+1] + sentence[i+2] + sentence[i+3]
        c = c.lower()
        if c[0] in alphabet and c[1] in alphabet and c[2] in alphabet and c[3] in alphabet:
            alpha4_freq[c] += 1

alpha4_top = set(sorted(alpha4_freq.keys(), key = lambda x : alpha4_freq[x], reverse=True)[:top_num*26])
alpha4_top

In [ ]:
sorted(alpha4_freq.keys(), key = lambda x : alpha4_freq[x], reverse=True)[2599]

In [ ]:
def transform(text, from_, to_):
    ret = ''
    dic = {}
    for i in range(len(from_)):
        dic[from_[i]] = to_[i]
    
    for i in text:
        isupper = i.isupper()
        i = i.lower()
        trans = i
        if dic.get(i) != None:
            trans = dic.get(i)
        if isupper:
            trans = trans.upper()
        ret += trans
    return ret

def score3(text):
    score = 0
    for i in range(len(text)-2):
        c = text[i] + text[i+1] + text[i+2]
        c = c.lower()
        if c in alpha3_top:
            score += 1
    return score

def score4(text):
    score = 0
    for i in range(len(text)-3):
        c = text[i] + text[i+1] + text[i+2] + text[i+3]
        c = c.lower()
        if c in alpha4_top:
            score += 1
    return score

def word_count(text):
    words = re.findall("[a-zA-Z]+", text)
    cnt = 0
    for word in words:
        word = word.lower()
        if word in word_top:
            cnt += 1
    return cnt

def weak_shuffle(list_, std=1):
    list_ = list_[:]
    size = len(list_)
    normal = np.random.normal(0, std, size)
    normal = normal.astype('int32')
    normal = list(normal)
    for i in range(size):
        idx = i
        idx += normal[i]
        idx = max([0, idx])
        idx = min([size-1, idx])
        tmp = list_[i]
        list_[i] = list_[idx]
        list_[idx] = tmp
    return list_

def incode(text):
    shuffled_alpha = [*alphabet]
    random.shuffle(shuffled_alpha)
    text = transform(t, alphabet, shuffled_alpha)
    return text

In [ ]:
t = input('암호화할 문장을 입력하세요: ')
incode(t)

In [ ]:
t = input('해독할 문장을 입력하세요: ')

t_freq = {}

for i in alphabet:
    t_freq[i] = 0

for i in t:
    i = i.lower()
    if alphabet.find(i) != -1:
        t_freq[i] += 1

t_rank = sorted(t_freq.keys(), key = lambda x : t_freq[x], reverse=True)

t = transform(t, t_rank, alpha_rank)

attempts = 1000
repeats = 10
max_score = score3(t) + score4(t)
best_t = t

score_trace, word_trace = [], []

print('\nUpdating Score')

for repeat in range(repeats):
    #if repeat == 1:
        #attempts //= 10
    for attempt in range(attempts):
        std = random.uniform(0.5, 4)
        shuffled = weak_shuffle(alpha_rank, std)
        trans_t = transform(t, alpha_rank, shuffled)
        t_score = score3(trans_t) + score4(trans_t)

        if t_score > max_score:
            max_score = t_score
            best_t = trans_t

            score_trace.append(max_score)
            word_trace.append(word_count(best_t))
    t = best_t

    print('\r{}/{}'.format(repeat+1, repeats), end='')

print('\n\nUpdating Word Count')

max_word = word_count(t)

for repeat in range(repeats):
    for i in range(26):
        for j in range(26):
            trans_t = transform(t, alphabet[i] + alphabet[j], alphabet[j] + alphabet[i])
            t_word = word_count(trans_t)
            if t_word > max_word:
                max_word = t_word
                best_t = trans_t
                t = best_t

                score_trace.append(score3(best_t) + score4(best_t))
                word_trace.append(max_word)

   

    print('\r{}/{}'.format(repeat+1, repeats), end='')

print()
print('\n'+t)
print()

plt.plot(score_trace, label = 'Score')
plt.plot(word_trace, label = 'Word Count')
plt.legend()
plt.show()

k

In [ ]:
words[:10000]

In [ ]:
alpha_rank

In [ ]:
weak_shuffle(alpha_rank, 1)

In [ ]:
print('\n\nUpdating Word Count')
t = 'Sdthough not the expdilit gosd, the keat alienle lsn resddy ke aeen sa refining ignorsnle. Alientiata, eapelisddy young onea, lsn get too okaeaaec with reaudta. Aoliety hedpa them sdong in thia msc lhsae. Kig cialoveriea sre loverec in the preaa, ahow up on the univeraity’a home psge, hedp get grsnta, snc msze the lsae for promotiona. Kut it’a wrong. Grest alientiata, the pioneera thst we scmire, sre not lonlernec with reaudta kut with the next jueationa. The highdy reapeltec phyailiat Enrilo Fermi todc hia atucenta thst sn experiment thst aulleaafuddy provea s hypotheaia ia s mesaurement; one thst coean’t ia s cialovery. S cialovery, sn unlovering ― of new ignorsnle. The Noked Pribe, the pinnslde of alientifil sllompdiahment, ia swsrcec, not for s difetime of alientifil slhievement, kut for s aingde cialovery, s reaudt. Even the Noked lommittee resdibea in aome wsy thst thia ia not resddy in the alientifil apirit, snc their swsrc litstiona lommondy honor the cialovery for hsving “openec s fiedc up,” “trsnaformec s fiedc,” or “tszen s fiedc in new snc unexpeltec cireltiona.”'
max_word = word_count(t)

for repeat in range(repeats):
    for i in range(26):
        for j in range(26):
            trans_t = transform(t, alphabet[i] + alphabet[j], alphabet[j] + alphabet[i])
            t_word = word_count(trans_t)
            if t_word > max_word:
                max_word = t_word
                best_t = trans_t
                t = best_t
                print(alphabet[i])
                print(alphabet[j])
                print(t)
                print(t_word)
                score_trace.append(score3(best_t) + score4(best_t))
                word_trace.append(max_word)
    

   

    print('\r{}/{}'.format(repeat+1, repeats), end='')

print()
print('\n'+t)
print()

plt.plot(score_trace, label = 'Score')
plt.plot(word_trace, label = 'Word Count')
plt.legend()
plt.show()

In [ ]:
t = 'Sdthough not the expdilit gosd, the keat alienle lsn resddy ke aeen sa refining ignorsnle. Alientiata, eapelisddy young onea, lsn get too okaeaaec with reaudta. Aoliety hedpa them sdong in thia msc lhsae. Kig cialoveriea sre loverec in the preaa, ahow up on the univeraity’a home psge, hedp get grsnta, snc msze the lsae for promotiona. Kut it’a wrong. Grest alientiata, the pioneera thst we scmire, sre not lonlernec with reaudta kut with the next jueationa. The highdy reapeltec phyailiat Enrilo Fermi todc hia atucenta thst sn experiment thst aulleaafuddy provea s hypotheaia ia s mesaurement; one thst coean’t ia s cialovery. S cialovery, sn unlovering ― of new ignorsnle. The Noked Pribe, the pinnslde of alientifil sllompdiahment, ia swsrcec, not for s difetime of alientifil slhievement, kut for s aingde cialovery, s reaudt. Even the Noked lommittee resdibea in aome wsy thst thia ia not resddy in the alientifil apirit, snc their swsrc litstiona lommondy honor the cialovery for hsving “openec s fiedc up,” “trsnaformec s fiedc,” or “tszen s fiedc in new snc unexpeltec cireltiona.”'
t=transform(t, 'ab', 'ba')
word_count(t)

In [ ]:
transform('Sdthough not the expdilit gosd, the keat alienle lsn resddy ke aeen sa refining ignorsnle', 'ab', 'ba')

In [ ]:
transform('Sdthough not', 'ab', 'ba')